# Kladd for sentimentanalyse

In [3]:
import dhlab as dh
import pandas as pd

wordcloud er ikke installert, kan ikke lage ordskyer


In [4]:
# Data import 
data_dir = "../norsentlex/Fullform/"
positive_tokens = pd.read_csv(data_dir + "/Fullform_Positive_lexicon.txt", names=["terms"])
negative_tokens = pd.read_csv(data_dir + "/Fullform_Negative_lexicon.txt", names=["terms"])

In [5]:
positive_tokens

,terms
0,absolutt
1,absolutta
2,absolutte
3,absoluttene
4,absolutter
...,...
6098,ønskete
6099,ønskt
6100,ønskte
6101,øyeblikkelig


In [6]:
negative_tokens

,terms
0,abnorm
1,abnorme
2,abnormt
3,abort
4,aborten
...,...
14834,ørlille
14835,ørlita
14836,ørlite
14837,ørliten


In [7]:
def count_sentiment_tokens(coll, terms):     
    
    target_terms = terms.join(coll, how="inner", on="terms")
    
    return target_terms

In [43]:
def group_index_terms(df):
    df.index = df.index.str.lower()
    df.index.name = "index"
    df = df.groupby('index')["counts"].sum().to_frame("counts")    
    assert df.index.is_unique
    return df

In [69]:
def label_sentiment_terms(coll, pos, neg):
    coll = group_index_terms(coll) 

    positive_counts = count_sentiment_tokens(coll, positive_tokens)
    negative_counts = count_sentiment_tokens(coll, negative_tokens)
    duplicates = pd.concat([coll, negative_counts, positive_counts], axis=1).index.drop_duplicates(keep=False).to_series(name="terms")
    neutral_counts = coll.join(duplicates, how="inner").reset_index(drop=True)

    positive_counts["sentiment"] = "pos"
    negative_counts["sentiment"] = "neg"
    neutral_counts["sentiment"] = "neut"

    sentlabels = pd.concat([positive_counts, negative_counts, neutral_counts], axis=0, ignore_index=True)
    
    return sentlabels

In [11]:
c = dh.Corpus(doctype="digavis", limit=10000)

In [104]:
coll = c.coll("barnevernet").frame

In [70]:
sentlabels = label_sentiment_terms(coll, positive_tokens, negative_tokens)

In [71]:
sentlabels

,terms,counts,sentiment
0,allsidig,1,pos
1,alvor,3,pos
2,alvoret,1,pos
3,alvorlig,7,pos
4,alvorlige,12,pos
...,...,...,...
5848,–,44,neut
5849,—,24,neut
5850,’,2,neut
5851,•,10,neut


In [113]:
sentlabels.counts.sum()



25451

In [107]:
len(coll.index)

5414

In [114]:

coll.counts.sum()



22560